In [7]:
# %% [markdown]
# ## 🚀 SFO 프로젝트 -  하이퍼파라미터 튜닝
#
# - **전처리 완료된 CSV 파일 로드**
# - Train/Validation 데이터 분할 (시간 기반)
# - **LightGBM 최종 모델 학습** (Optuna 최적 파라미터 조회)
# - **검증 데이터에 대한 예측 및 MAE, R2 출력**
# - **학습된 모델 파일로 저장**
#
# **[시작 전 준비사항]**
# 1. `02_time_series(feature_engineering).ipynb` 스크립트를 먼저 실행하여 `time_series.csv` 파일이 생성되어 있어야 합니다.
# 2. 필요한 라이브러리 설치: `pip install pandas numpy lightgbm scikit-learn joblib`
#

# %%
import pandas as pd
import numpy as np
import os
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, r2_score
import joblib # 모델 저장을 위한 라이브러리

# --- 0. 설정 및 파일 경로 정의 ---
input_processed_csv_file = '../data/time_series.csv' # 중요!! 경로 설정해줘야함!!
model_save_path = '../data/optuna_lightgbm_model.pkl' # 학습된 모델 저장 경로

target_column = 'T일 예정 수주량'

print("--- [SFO 프로젝트 - 하이퍼파라미터 튜닝] ---")


# --- 1단계: 피처엔지니어링 데이터 로드 ---
print(f"\n--- [1단계: 피처엔지니어링 데이터 '{input_processed_csv_file}' 로드 시작] ---")
df_manual_features_encoded = None
try:
    if not os.path.exists(input_processed_csv_file):
        raise FileNotFoundError(f"🚨 오류: 입력 파일 '{input_processed_csv_file}'을(를) 찾을 수 없습니다. \n   [파일 1: 01_preprocess_and_save.py] 스크립트를 먼저 실행했는지 확인하세요.")
    
    df_manual_features_encoded = pd.read_csv(input_processed_csv_file)
    
    # !! 매우 중요 !!
    # CSV로 저장하면서 문자열이 된 'Date' 컬럼을 다시 datetime 객체로 변환합니다.
    # 시간 기반 데이터 분할(splitting)을 위해 필수입니다.
    df_manual_features_encoded['Date'] = pd.to_datetime(df_manual_features_encoded['Date'])
    
    print(f"✅ 전처리된 데이터 로드 완료 (shape: {df_manual_features_encoded.shape})")
    print(f"   'Date' 컬럼 타입: {df_manual_features_encoded['Date'].dtype}")

except FileNotFoundError as e:
    print(f"🚨 {e}")
except Exception as e:
    print(f"🚨 1단계 데이터 로드 중 오류: {e}")
    df_manual_features_encoded = None


--- [SFO 프로젝트 - 하이퍼파라미터 튜닝] ---

--- [1단계: 피처엔지니어링 데이터 '../data/time_series.csv' 로드 시작] ---
✅ 전처리된 데이터 로드 완료 (shape: (10624, 49))
   'Date' 컬럼 타입: datetime64[ns]


In [8]:

# 2.1. 데이터 분할 (Train/Validation - 시간 기반)
# 'Date' 컬럼이 datetime 타입이어야 sort_values가 올바르게 작동합니다.
df_manual_features_encoded = df_manual_features_encoded.sort_values(by='Date')

train_ratio = 0.8
split_idx = int(len(df_manual_features_encoded) * train_ratio)

train_df = df_manual_features_encoded.iloc[:split_idx]
val_df = df_manual_features_encoded.iloc[split_idx:]

# ▼▼▼▼▼ [수정된 부분] ▼▼▼▼▼
# 'object' 타입의 원본 Product_Number 컬럼을 피처에서 제외시킵니다.
features_to_exclude_from_X = ['Date', target_column, 'Product_Number']
# ▲▲▲▲▲ [수정된 부분] ▲▲▲▲▲

X_train = train_df.drop(columns=features_to_exclude_from_X)
y_train = train_df[target_column]
X_val = val_df.drop(columns=features_to_exclude_from_X)
y_val = val_df[target_column]

# (참고) 만약 'DayOfWeek' 원본 컬럼도 남아있다면 위 리스트에 'DayOfWeek'도 추가해야 할 수 있습니다.
# features_to_exclude_from_X = ['Date', target_column, 'Product_Number', 'DayOfWeek']

print(f"2.1. 데이터 분할 완료: Train_X: {X_train.shape}, Train_y: {y_train.shape}, Val_X: {X_val.shape}, Val_y: {y_val.shape}")
print(f"   - 최종 모델 학습에 사용될 피처 개수: {X_train.shape[1]}개")

2.1. 데이터 분할 완료: Train_X: (8499, 46), Train_y: (8499,), Val_X: (2125, 46), Val_y: (2125,)
   - 최종 모델 학습에 사용될 피처 개수: 46개


In [9]:
# %%
# --- 02_Hyperparameter_Tuning.ipynb ---
import optuna # Optuna 임포트

print("--- [02_Hyperparameter_Tuning (Optuna) 시작] ---")
categorical_features = [
    'Month', 'Day', 'DoW', 'IsWeekend', 
    'Year', 'Quarter'
]
categorical_features_final = [col for col in categorical_features if col in X_train.columns]
# --- Optuna 목적 함수 정의 (Objective Function) ---
print("\n--- [2. Optuna 목적 함수 정의] ---")

def objective(trial):
    # LightGBM 하이퍼 파라미터 탐색 공간 정의
    params = {
        'objective': 'regression_l1', # MAE
        'metric': 'mae',
        'n_estimators': 2000, # early_stopping을 사용하므로 크게 설정
        'n_jobs': -1,
        'random_state': 42,
        'verbose': -1,
        'boosting_type': 'gbdt',
        
        # 튜닝 대상 파라미터
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.05),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0, step=0.05),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
    }

    # 모델 생성 및 학습
    model = lgb.LGBMRegressor(**params)
    
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='mae',
              callbacks=[lgb.early_stopping(100, verbose=False)],
              categorical_feature=categorical_features_final
             )
    
    # Validation MAE 반환
    val_mae = model.best_score_['valid_0']['l1']
    return val_mae

print("  'objective' 함수 정의 완료.")


# %%
# --- 3. Optuna 스터디 실행 ---
if X_train is not None:
    print("\n--- [3. Optuna 스터디 실행 (튜닝 시작)] ---")
    
    # 'minimize' (MAE 최소화) 방향으로 스터디 생성
    study = optuna.create_study(direction='minimize')
    
    # 튜닝 실행 (n_trials 횟수 조절 가능, 50~100회 권장)
    study.optimize(objective, n_trials=50) # 50회 시도

    print("\n--- [튜닝 완료] ---")
    print(f"  최적의 MAE: {study.best_value:.4f}")
    print("  최적의 하이퍼 파라미터:")
    print(study.best_params)
else:
    print("🚨 데이터가 로드되지 않아 튜닝을 스킵합니다.")


# %%
# --- 4. (권장) 최적 파라미터로 최종 모델 학습 및 저장 ---
if 'study' in locals() and study.best_params:
    print("\n--- [4. 최적 파라미터로 최종 모델 학습 및 저장] ---")
    
    # 기본 파라미터에 best_params를 업데이트
    final_params = {
        'objective': 'regression_l1',
        'metric': 'mae',
        'n_estimators': 2000, # early_stopping으로 조절됨
        'n_jobs': -1,
        'random_state': 42,
        'verbose': -1,
        'boosting_type': 'gbdt',
    }
    
    final_params.update(study.best_params) # 튜닝된 파라미터 덮어쓰기
    
    final_model = lgb.LGBMRegressor(**final_params)
    
    print("  최종 모델 학습 시작...")
    final_model.fit(X_train, y_train,
                    eval_set=[(X_val, y_val)],
                    eval_metric='mae',
                    callbacks=[lgb.early_stopping(100, verbose=False)], # [수정] 출력이 겹치지 않게 verbose=False로
                    categorical_feature=categorical_features_final
                   )
    
    # --- [수정된 출력 부분] ---
    
    # 1. R2 계산을 위해 예측 수행
    val_preds = final_model.predict(X_val)
    
    # 2. R2 스코어 계산
    final_r2 = r2_score(y_val, val_preds)
    
    # 3. MAE 스코어 가져오기 (학습 중 계산된 best score)
    final_mae = final_model.best_score_['valid_0']['l1']
    
    # 4. 모델 저장
    joblib.dump(final_model, model_save_path)
    
    # 5. 요청하신 형식으로 출력
    print("\n✅ LightGBM 최종 모델 학습 완료!")
    print(f" - 최종 모델 검증 MAE: {final_mae:.2f}") # 소수점 2자리 (15.19)
    print(f" - 최종 모델 검증 R2: {final_r2:.3f}")   # 소수점 3자리 (0.940)
    print(f"✅ 학습된 최종 모델이 '{model_save_path}'에 저장되었습니다.")
    # --- [출력 수정 완료] ---

else:
    print("🚨 Optuna 튜닝이 실행되지 않아 최종 모델을 저장할 수 없습니다.")

print("\n--- [02_Hyperparameter_Tuning (Optuna) 종료] ---")

[I 2025-11-12 16:15:45,929] A new study created in memory with name: no-name-692c7dcd-7c11-4ca6-8a79-6899e40a4699


--- [02_Hyperparameter_Tuning (Optuna) 시작] ---

--- [2. Optuna 목적 함수 정의] ---
  'objective' 함수 정의 완료.

--- [3. Optuna 스터디 실행 (튜닝 시작)] ---


[I 2025-11-12 16:16:05,844] Trial 0 finished with value: 11.893004248837563 and parameters: {'learning_rate': 0.018777171659784965, 'num_leaves': 60, 'max_depth': 15, 'subsample': 0.65, 'colsample_bytree': 0.75, 'min_child_samples': 20, 'reg_alpha': 0.42191961730425726, 'reg_lambda': 0.23992965724240928}. Best is trial 0 with value: 11.893004248837563.
[I 2025-11-12 16:16:11,334] Trial 1 finished with value: 13.786293500510533 and parameters: {'learning_rate': 0.024375165454474226, 'num_leaves': 77, 'max_depth': 5, 'subsample': 0.8, 'colsample_bytree': 0.85, 'min_child_samples': 43, 'reg_alpha': 0.8079711008146012, 'reg_lambda': 0.18997340470883398}. Best is trial 0 with value: 11.893004248837563.
[I 2025-11-12 16:16:22,677] Trial 2 finished with value: 12.687573015951196 and parameters: {'learning_rate': 0.01703263162041868, 'num_leaves': 46, 'max_depth': 14, 'subsample': 0.8, 'colsample_bytree': 0.6, 'min_child_samples': 46, 'reg_alpha': 0.39427554353726907, 'reg_lambda': 0.407873396


--- [튜닝 완료] ---
  최적의 MAE: 10.9892
  최적의 하이퍼 파라미터:
{'learning_rate': 0.027719430829110773, 'num_leaves': 100, 'max_depth': 11, 'subsample': 0.6, 'colsample_bytree': 0.7, 'min_child_samples': 11, 'reg_alpha': 0.42067115008505107, 'reg_lambda': 0.05110000164534667}

--- [4. 최적 파라미터로 최종 모델 학습 및 저장] ---
  최종 모델 학습 시작...

✅ LightGBM 최종 모델 학습 완료!
 - 최종 모델 검증 MAE: 10.99
 - 최종 모델 검증 R2: 0.972
✅ 학습된 최종 모델이 '../data/optuna_lightgbm_model.pkl'에 저장되었습니다.

--- [02_Hyperparameter_Tuning (Optuna) 종료] ---
